In [1]:
pip install pyspark

In [2]:
pip install faker

In [3]:
from faker import Faker
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField,StringType, IntegerType, FloatType, DateType
import numpy as np
import pandas as pd
import random
import datetime
from dateutil.relativedelta import relativedelta

In [4]:
# Создание SparkSession
spark = SparkSession.builder \
.appName("SparkDataGnrt") \
.getOrCreate()

In [5]:
def df_create(num):
  fake = Faker("ru_RU")
  end_date = datetime.date.today()
  st_date = end_date - relativedelta(years=1)
  data = []
  for _ in range(num):
    data.append({
        'date': fake.date_between(st_date, end_date),
        'user_id': fake.uuid4(),
        'product': fake.random_element(['food', 'medicines', 'cosmetics', 'clothes', 'instrument']),
        'quantity': random.randint(1, 100),
        'price': random.uniform(10, 1000),

    })
  df = pd.DataFrame(data)
  return df

In [6]:
# Определяем структуру данных
my_schema = StructType([
StructField("date", DateType(), True),
StructField("user_id", StringType(), True),
StructField("product", StringType(), True),
StructField("quantity", IntegerType(), True),
StructField("price", FloatType(), True)                         ])

In [7]:
# Создание DataFrame
df_spark = spark.createDataFrame(df_create(1000), schema= my_schema)


In [8]:
df_spark = df_spark.repartition(1)
df_spark.write.option("header",True) \
 .csv("/Загрузки/df_spark")

In [9]:
# Закрытие SparkSession
spark.stop()